# Designing the Software Architecture
## Data Processing Module
Handles data preprocessing, including missing values, feature engineering, and scaling.
This module should be independent, so it can be reused or updated without affecting other parts of the system.

## Model Inference Module
Loads the pre-trained Random Forest model and performs predictions.
This module should be optimized for scalability, enabling multiple requests to be processed simultaneously.

## Error Handling Module
Manages exceptions and errors throughout the system, ensuring graceful degradation.
Provides clear error messages to the API users.

## REST API
The API should accept input features in JSON format and return the predicted housing price.
It will include endpoints for predictions and possibly for model health checks.

## High-Level Architecture:
Data Input (JSON) → Data Processing Module → Model Inference Module → Error Handling Module → API Response (JSON)

## Implementing the Architecture in Python

## Data Processing Module

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

class DataProcessor:
    def __init__(self):
        self.scaler = StandardScaler()
    
    def preprocess(self, data):
        # Assuming `data` is a dictionary with input features
        df = pd.DataFrame([data])
        
        # Fill missing values (example strategy)
        df.fillna(df.mean(), inplace=True)
        
        # Feature scaling (example on all numeric columns)
        df[df.columns] = self.scaler.fit_transform(df[df.columns])
        
        return df


## Model Inference Module

In [ ]:
import joblib

class ModelInference:
    def __init__(self, model_path):
        self.model = joblib.load(model_path)
    
    def predict(self, processed_data):
        return self.model.predict(processed_data)


## Error Handling Module

In [ ]:
class ErrorHandler:
    def __init__(self):
        pass
    
    def handle_error(self, error):
        # Log the error or handle it as needed
        return {"error": str(error)}


# Implementing the REST API using FastAPI
## Setting Up FastAPI

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

# Load the components
data_processor = DataProcessor()
model_inference = ModelInference(model_path='random_forest_model.pkl')
error_handler = ErrorHandler()

class HousingData(BaseModel):
    longitude: float
    latitude: float
    housing_median_age: float
    total_rooms: float
    total_bedrooms: float
    population: float
    households: float
    median_income: float
    ocean_proximity: str  # Assuming ocean proximity is categorical

@app.post("/predict/")
async def predict_price(data: HousingData):
    try:
        # Convert incoming data to dictionary
        input_data = data.dict()
        
        # Process the data
        processed_data = data_processor.preprocess(input_data)
        
        # Perform prediction
        prediction = model_inference.predict(processed_data)
        
        return {"predicted_price": prediction[0]}
    except Exception as e:
        # Handle error
        return error_handler.handle_error(e)

# Example endpoint to check API status
@app.get("/")
def read_root():
    return {"message": "Housing Price Prediction API is running"}


# Explanation
## Modular Design:
Each module (data processing, inference, error handling) is independent and can be tested or updated separately.

## Scalable Architecture:
The modularity and use of FastAPI ensure the system can handle multiple requests and scale as needed.

## API Design:
The REST API provides a simple interface for external systems to interact with your model, with built-in error handling.